In [21]:
import json
import pandas as pd
import paho.mqtt.client as mqtt
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import paho.mqtt.client as mqtt
import json
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# DataFrame to store last 6 records
columns = ["Glucose", "BloodPressure", "Insulin", "Outcome"]
data = pd.DataFrame(columns=columns)

correct_classifications = 0
total_records = 0

def on_connect(client, userdata, flags, rc):
    print("Connected with result code", rc)
    client.subscribe("myroom/jacob")

def on_message(client, userdata, msg):
    global data, correct_classifications, total_records

    if total_records >= 1000:
        return

    # Load the JSON data from the message
    record = json.loads(msg.payload.decode())
    new_data = pd.DataFrame([[record["Glucose"], record["BloodPressure"], record["Insulin"], record["Outcome"]]], columns=columns)

    # Append and keep only last 6 records
    data = pd.concat([data, new_data], ignore_index=True).tail(6)
    
    #print(f"Received record:\n{record}")

    # Wait until we have 6 records before classifying
    if len(data) == 6:
        X_train = data.iloc[:-1, :-1].values  # First 5 records as training data
        y_train = data.iloc[:-1, -1].values   # First 5 labels
        X_test = data.iloc[-1, :-1].values.reshape(1, -1)  # Last record as test data
        y_test = data.iloc[-1, -1]  # True label for test data

        knn = KNeighborsClassifier(n_neighbors=3)
        knn.fit(X_train, y_train)
        prediction = knn.predict(X_test)

        if prediction == y_test:
            correct_classifications += 1

        total_records += 1

    # Print progress every 10 records
    if total_records % 10 == 0:
        print(f"Progress: {total_records} records processed")

    # Stop after 1000 records
    if total_records >= 1000:
        print(f"Completed 1000 records. Correct classifications: {correct_classifications}")
        client.disconnect()

# Create and configure MQTT client
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

# Connect to MQTT broker
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# Start listening for messages
mqttc.loop_forever()


Connected with result code 0
Progress: 0 records processed
Progress: 0 records processed
Progress: 0 records processed
Progress: 0 records processed
Progress: 0 records processed


C:\Users\Jacob\AppData\Local\Temp\ipykernel_9876\3873552299.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, new_data], ignore_index=True).tail(6)


Progress: 10 records processed
Progress: 20 records processed
Progress: 30 records processed
Progress: 40 records processed
Progress: 50 records processed
Progress: 60 records processed
Progress: 70 records processed
Progress: 80 records processed
Progress: 90 records processed
Progress: 100 records processed
Progress: 110 records processed
Progress: 120 records processed
Progress: 130 records processed
Progress: 140 records processed
Progress: 150 records processed
Progress: 160 records processed
Progress: 170 records processed
Progress: 180 records processed
Progress: 190 records processed
Progress: 200 records processed
Progress: 210 records processed
Progress: 220 records processed
Progress: 230 records processed
Progress: 240 records processed
Progress: 250 records processed
Progress: 260 records processed
Progress: 270 records processed
Progress: 280 records processed
Progress: 290 records processed
Progress: 300 records processed
Progress: 310 records processed
Progress: 320 rec

7